## Issue Credential

Issuing a credential involves establishing a connection between the issuer and the holder, which is done by following the process outlined in `Example 01 - Connections`. Once the connection is established, the issuer will prepare and send a credential offer, which creates an issue record on both the issuer's and holder's agents. The holder will then retrieve the list of issue records, find the one they wish to accept, and notify the issuer of their acceptance. Finally, the issuer will issue the credential to the holder, completing the process.

This example presents the steps required to issue a credential using Atala PRISM Agents.

In [1]:
#🚨 Run this code cell to import requirements in the Kernel

import os
import time
import datetime
import base64
from pprint import pprint
from dotenv import load_dotenv
from termcolor import colored,cprint

from prism_agent_open_api_specification_client import Client
from prism_agent_open_api_specification_client.types import Response
from prism_agent_open_api_specification_client.models import ErrorResponse
from prism_agent_open_api_specification_client.models import Connection,ConnectionInvitation,CreateConnectionRequest,AcceptConnectionInvitationRequest
from prism_agent_open_api_specification_client.api.connections_management import get_connection,create_connection,accept_connection_invitation
from prism_agent_open_api_specification_client.models import IssueCredentialRecord, CreateIssueCredentialRecordRequest, IssueCredentialRecordBaseClaims, IssueCredentialRecordCollection, IssueCredentialRecordAllOfProtocolState
from prism_agent_open_api_specification_client.api.issue_credentials_protocol import get_credential_record, get_credential_records, create_credential_offer,accept_credential_offer,issue_credential
from prism_agent_open_api_specification_client.models import CreateManagedDidRequest, CreateManagedDIDResponse
from prism_agent_open_api_specification_client.models import ListManagedDIDResponseInner
from prism_agent_open_api_specification_client.models import DIDOperationResponse, DIDResponse, DID, Service   
from prism_agent_open_api_specification_client.api.did_registrar import create_managed_did, publish_managed_did
from prism_agent_open_api_specification_client.api.did import get_did

### Ultilitary functions

In [2]:
def get_invitation_str(connection):
    parts = connection.invitation.invitation_url.split("=")
    return parts[1]

def find_credential_record_by_state(client, state):
    credential_records: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=client)

    for offer in credential_records.contents:
        if(offer.protocol_state == state):
            return offer
    return None 

def print_credential_record(credential_record):
    print(f"record_id:          {credential_record.record_id}")
    print(f"subject_id:         {credential_record.subject_id}")
    print(f"role:               {credential_record.role}")
    print(f"protocol_state:     {credential_record.protocol_state}")
    print(f"created_at:         {credential_record.created_at}")
    print(f"updated_at:         {credential_record.updated_at}")
    
def print_connection(connection):
    print(f"connection_id: {connection.connection_id}")
    print(f"state:         {connection.state}")
    print(f"label:         {connection.label}")
    print(f"my_did:        {connection.my_did}")
    print(f"their_did:     {connection.their_did}")
    print(f"created_at:    {connection.created_at}")

### Client instances

We will create two clients, one for the Issuer and one for the Holder, to establish a connection and perform the issue credential process.

⚠️ Remember to update the file variables.env with the URLs and API keys provided to you.


In [3]:
load_dotenv("../BetaProgram/variables.env")
issuerApiKey = os.getenv('ISSUER_APIKEY')
issuerUrl = os.getenv('ISSUER_URL')

holderApiKey = os.getenv('HOLDER_APIKEY')
holderUrl = os.getenv('HOLDER_URL')

issuer_client = Client(base_url=issuerUrl, headers={"apiKey": issuerApiKey})
holder_client = Client(base_url=holderUrl, headers={"apiKey": holderApiKey})

print(f"Issuer URL:{issuerUrl}")
print(f"Holder URL:{holderUrl}")

Issuer URL:http://host.docker.internal:8080/prism-agent
Holder URL:http://host.docker.internal:8090/prism-agent


### Create connection

ℹ️ For details on this see "Example 01 - Connections"

In [4]:
print("Please wait...")

conn_request = CreateConnectionRequest()
conn_request.label = f'Issue credential {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
issuer_connection: Response[Connection] =  create_connection.sync(client=issuer_client,json_body=conn_request)

invitation = get_invitation_str(issuer_connection)

accept_conn_request = AcceptConnectionInvitationRequest(invitation)
holder_connection: Response[ConnectionInvitation] =  accept_connection_invitation.sync(client=holder_client,json_body=accept_conn_request)


issuer_connection: Response[Connection] = get_connection.sync(client=issuer_client,connection_id=issuer_connection.connection_id)
holder_connection: Response[Connection] = get_connection.sync(client=holder_client,connection_id=holder_connection.connection_id)

print(f"Issuer connection: {issuer_connection.connection_id}")
print(f"Holder connection: {holder_connection.connection_id}\n")

while (issuer_connection.state != 'ConnectionResponseSent' or 
       not(holder_connection.state == 'ConnectionResponseReceived' or holder_connection.state == 'ConnectionRequestSent')):
    issuer_connection: Response[Connection] = get_connection.sync(client=issuer_client,connection_id=issuer_connection.connection_id)
    holder_connection: Response[Connection] = get_connection.sync(client=holder_client,connection_id=holder_connection.connection_id)
    print("Issuer State: {} / Holder State: {}".format(issuer_connection.state,holder_connection.state))
    time.sleep(1)
    
print("Connection established between Issuer and Holder!")
print("\nIssuer connection:\n")
print_connection(issuer_connection)
print("\nHolder connection:\n")
print_connection(holder_connection)

Please wait...
Issuer connection: 8a1c353d-ebcc-4783-9f8e-0abbfc4affae
Holder connection: c38acdae-c07e-4812-8808-fbc1da82fbba

Issuer State: InvitationGenerated / Holder State: ConnectionRequestPending
Issuer State: ConnectionResponsePending / Holder State: ConnectionResponseReceived
Issuer State: ConnectionResponsePending / Holder State: ConnectionResponseReceived
Issuer State: ConnectionResponseSent / Holder State: ConnectionResponseReceived
Connection established between Issuer and Holder!

Issuer connection:

connection_id: 8a1c353d-ebcc-4783-9f8e-0abbfc4affae
state:         ConnectionResponseSent
label:         Issue credential 2023-03-07 18:27:16
my_did:        did:peer:2.Ez6LSch17z4mJpDan2jMouDTct5KuDBfeHHLC4PE58UvgHcQ7.Vz6MkuW5jdg67iAhZoc6H7J2kKz5c4FbToqUa29S4R3FMrRsn.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwODAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0
their_did:     did:peer:2.Ez6LSm288ZWXYtHiWGZAsTi5PCJgqUzceRXUpGHw9DZUdF43Y.Vz6MkkVVakURckQDYQxZBSV

### Holder - Create an unpublished did:prism 

To issue a verifiable credential, the Holder must provide the credential subject to the Issuer. The credential subject is a DID identifier. We will use a long-form `did:prism` in this example. So the next step is to create an unpublished `did` on the Holder side. It is required to have a key with `purpose` equal to `assertionMethod` in the DID Document template.

ℹ️ For details on creating an unpublished `did` see "Example 02 - DID Registrar"

In [5]:
data = { 
    "documentTemplate": {
        "publicKeys": [
            {
                "id": "key1",
                "purpose": "authentication"
            },
            {
                "id": "key2",
                "purpose": "assertionMethod"
            }
        ],
        "services": [
            {
                "id": "did:prism:test1",
                "type": "LinkedDomains",
                "serviceEndpoint": [
                    "https://test1.com"
                ]
            }
        ]
    }
}

did_request = CreateManagedDidRequest.from_dict(data)
subject_did: Response[CreateManagedDIDResponse] = create_managed_did.sync(client=holder_client, json_body=did_request)

print(f"Subject did (credential subject): {subject_did.long_form_did}")

Subject did (credential subject): did:prism:3b3cf4fc89942db5e84c4f4774c977368a8382f02d95b3f8ff6ff1bbdd44a4a6:Cs0CCsoCElkKBGtleTEQBEJPCglzZWNwMjU2azESILV-iwuC0ZbQCo4l0uJRGNhQ413e4YH2kkZYWaEseruLGiAooIpYsSFC3aTp7MwDdICS6JUw61vMzOZirbJOjUrbmBJZCgRrZXkyEAJCTwoJc2VjcDI1NmsxEiAcQuf8FYngKiJIvc1OxJ-BiuQ6h6n2OY2jRjx_YHWUoxogZB-tMdujutwiKKl5h_UpiuBuPu9xOxOhzh8PhXaVlx4SXAoHbWFzdGVyMBABQk8KCXNlY3AyNTZrMRIg3IHP3PMXes7WOiXbGt4i2xxmlt-YQxpSXLpxHvdwzmYaICjagobmCVU8J5YSYFjMNkDrUd3kaA2bTaZFHSU-wRCcGjQKD2RpZDpwcmlzbTp0ZXN0MRINTGlua2VkRG9tYWlucxoSaHR0cHM6Ly90ZXN0MS5jb20v


### Issuer - Create and publish a did:prism 

To issue a verifiable credential, the Issuer must provide the issuing DID. The issuing DID is a DID identifier. We will use a canonical-form `did:prism` in this example. So the next step is to create and publish a `did` on the Issuer side. It is required to have a key with `purpose` equal to `assertionMethod` in the DID Document template.

ℹ️ For details on creating and publishing a `did` see "Example 02 - DID Registrar"

In [6]:
# Reusing the same did documentTemplate
issuing_did: Response[CreateManagedDIDResponse] = create_managed_did.sync(client=issuer_client, json_body=did_request)

print(f"Issuing did: {issuing_did.long_form_did}")

operation_response : (DIDOperationResponse) = publish_managed_did.sync(client=issuer_client, did_ref=issuing_did.long_form_did)

print("Please wait...")

issuing_did : [DIDResponse] = get_did.sync(client=issuer_client, did_ref=operation_response.scheduled_operation.did_ref)

while (isinstance(issuing_did, ErrorResponse)):
    issuing_did : [DIDResponse] = get_did.sync(client=issuer_client, did_ref=operation_response.scheduled_operation.did_ref)
    print("Please wait...")
    time.sleep(10)
    
print(f"Issuing did published: {issuing_did.did.id}")

Issuing did: did:prism:ce87a140e0b43ba1feb9b55ece626bd2229e057907bd46c5f866b96738a3b2fe:Cs0CCsoCElkKBGtleTEQBEJPCglzZWNwMjU2azESIPFOjFxjVAbEXJy7YKm-bZXscT0JDNvQvtoc9GYK2kNAGiD_ZX6TNFJYNpPRF4eyd-BCy-jgVNpYTVg7LcJ0TldBcBJZCgRrZXkyEAJCTwoJc2VjcDI1NmsxEiBwRFiF_1M5PhFR9kL0Llyd21aBE5YZC_HIJAQf5hDDWBogdd7_f6KBvfz9r8UlK-6Ssek9DW-RfCXI3iTt8a7tVK4SXAoHbWFzdGVyMBABQk8KCXNlY3AyNTZrMRIg_xqjoZZEUPtU-6YBtZ55OsitDMsPPXDCjngxt1v9GHEaINeC_zLPq9lrxFWZRJxeAmp4V6OHTCTqDtwM3a8o1BRiGjQKD2RpZDpwcmlzbTp0ZXN0MRINTGlua2VkRG9tYWlucxoSaHR0cHM6Ly90ZXN0MS5jb20v
Please wait...
Please wait...
Please wait...
Please wait...
Issuing did published: did:prism:ce87a140e0b43ba1feb9b55ece626bd2229e057907bd46c5f866b96738a3b2fe


### Issuer - Create credential claim

Now that we have the DIDs ready, the next step is for the Issuer to create the credential claim object. The credential claim contains the attributes that will be part of the verifiable credential.

In [7]:
data = {
        "firstname": 'James',
        "lastname": 'Smith',
        "birthdate": '01/01/2000'
      }

credential_claims = IssueCredentialRecordBaseClaims().from_dict(data)

### Issuer - Send credential offer

Following, the Issuer creates a `CreateIssueCredentialRecordRequest`. It contains the `subject_id`, the `claims` and other metadata. This object is passed to the `create_credential_offer` endpoint.
The `create_credential_offer` call creates an `IssueCredentialRecord` on the issuer side, it also sends the credential offer to the Holder.

In [8]:
credential_offer = CreateIssueCredentialRecordRequest(subject_id=subject_did.long_form_did, 
                                                      issuing_did=issuing_did.did.id,
                                                      claims=credential_claims, 
                                                      connection_id=issuer_connection.connection_id,
                                                      validity_period=3600, 
                                                      automatic_issuance=False)

issuer_credential_record: Response[IssueCredentialRecord] = create_credential_offer.sync(client=issuer_client,json_body=credential_offer)

print("\nIssuer credential record:\n")
print_credential_record(issuer_credential_record)


Issuer credential record:

record_id:          b02cafe0-a8e0-402b-9398-d0a6c3909b61
subject_id:         did:prism:3b3cf4fc89942db5e84c4f4774c977368a8382f02d95b3f8ff6ff1bbdd44a4a6:Cs0CCsoCElkKBGtleTEQBEJPCglzZWNwMjU2azESILV-iwuC0ZbQCo4l0uJRGNhQ413e4YH2kkZYWaEseruLGiAooIpYsSFC3aTp7MwDdICS6JUw61vMzOZirbJOjUrbmBJZCgRrZXkyEAJCTwoJc2VjcDI1NmsxEiAcQuf8FYngKiJIvc1OxJ-BiuQ6h6n2OY2jRjx_YHWUoxogZB-tMdujutwiKKl5h_UpiuBuPu9xOxOhzh8PhXaVlx4SXAoHbWFzdGVyMBABQk8KCXNlY3AyNTZrMRIg3IHP3PMXes7WOiXbGt4i2xxmlt-YQxpSXLpxHvdwzmYaICjagobmCVU8J5YSYFjMNkDrUd3kaA2bTaZFHSU-wRCcGjQKD2RpZDpwcmlzbTp0ZXN0MRINTGlua2VkRG9tYWlucxoSaHR0cHM6Ly90ZXN0MS5jb20v
role:               Issuer
protocol_state:     OfferPending
created_at:         2023-03-07 18:27:51.910417+00:00
updated_at:         <prism_agent_open_api_specification_client.types.Unset object at 0x7f72c56ff1f0>


### Holder - Wait for credential offer

The Holder waits to receive the credential offer. When received, it will show up in the Holder's credential records list as a new entry with `protocol_state` equal to `OfferReceived`. The code below waits until a credential offer is received and takes the corresponding credential_record.

In [9]:
holder_credential_record = find_credential_record_by_state(holder_client, IssueCredentialRecordAllOfProtocolState.OFFERRECEIVED)

while(holder_credential_record == None):
    holder_credential_record = find_credential_record_by_state(holder_client, IssueCredentialRecordAllOfProtocolState.OFFERRECEIVED)
    time.sleep(1)
    
print("\nHolder credential record:\n")
print_credential_record(holder_credential_record)


Holder credential record:

record_id:          65601b27-13ad-4bec-8ba3-e5b51ee03f2e
subject_id:         did:prism:3b3cf4fc89942db5e84c4f4774c977368a8382f02d95b3f8ff6ff1bbdd44a4a6:Cs0CCsoCElkKBGtleTEQBEJPCglzZWNwMjU2azESILV-iwuC0ZbQCo4l0uJRGNhQ413e4YH2kkZYWaEseruLGiAooIpYsSFC3aTp7MwDdICS6JUw61vMzOZirbJOjUrbmBJZCgRrZXkyEAJCTwoJc2VjcDI1NmsxEiAcQuf8FYngKiJIvc1OxJ-BiuQ6h6n2OY2jRjx_YHWUoxogZB-tMdujutwiKKl5h_UpiuBuPu9xOxOhzh8PhXaVlx4SXAoHbWFzdGVyMBABQk8KCXNlY3AyNTZrMRIg3IHP3PMXes7WOiXbGt4i2xxmlt-YQxpSXLpxHvdwzmYaICjagobmCVU8J5YSYFjMNkDrUd3kaA2bTaZFHSU-wRCcGjQKD2RpZDpwcmlzbTp0ZXN0MRINTGlua2VkRG9tYWlucxoSaHR0cHM6Ly90ZXN0MS5jb20v
role:               Holder
protocol_state:     OfferReceived
created_at:         2023-03-07 18:27:52.752750+00:00
updated_at:         <prism_agent_open_api_specification_client.types.Unset object at 0x7f72c56ff1f0>


### Holder - Accept credential offer

Now the Holder uses the `accept_credential_offer` endpoint to accept the credential offer. It must provide the `record_id` of the offer. Accepting the credential offer tells the Issuer that the credential can be issued.

In [10]:
holder_credential_record: Response[IssueCredentialRecord] = accept_credential_offer.sync(client=holder_client, record_id=holder_credential_record.record_id)
print("\nHolder credential record:\n")
print_credential_record(holder_credential_record)


Holder credential record:

record_id:          65601b27-13ad-4bec-8ba3-e5b51ee03f2e
subject_id:         did:prism:3b3cf4fc89942db5e84c4f4774c977368a8382f02d95b3f8ff6ff1bbdd44a4a6:Cs0CCsoCElkKBGtleTEQBEJPCglzZWNwMjU2azESILV-iwuC0ZbQCo4l0uJRGNhQ413e4YH2kkZYWaEseruLGiAooIpYsSFC3aTp7MwDdICS6JUw61vMzOZirbJOjUrbmBJZCgRrZXkyEAJCTwoJc2VjcDI1NmsxEiAcQuf8FYngKiJIvc1OxJ-BiuQ6h6n2OY2jRjx_YHWUoxogZB-tMdujutwiKKl5h_UpiuBuPu9xOxOhzh8PhXaVlx4SXAoHbWFzdGVyMBABQk8KCXNlY3AyNTZrMRIg3IHP3PMXes7WOiXbGt4i2xxmlt-YQxpSXLpxHvdwzmYaICjagobmCVU8J5YSYFjMNkDrUd3kaA2bTaZFHSU-wRCcGjQKD2RpZDpwcmlzbTp0ZXN0MRINTGlua2VkRG9tYWlucxoSaHR0cHM6Ly90ZXN0MS5jb20v
role:               Holder
protocol_state:     RequestPending
created_at:         2023-03-07 18:27:52.752750+00:00
updated_at:         2023-03-07 18:27:54.163705+00:00


### Issuer - Wait for credential request

The Issuer waits to receive the credential request. When received, the credential record state will change to `RequestReceived` in the Issuer's credential records list. The code below waits until a credential request is received and takes the corresponding credential_record.

In [11]:
print("Please wait...")

issuer_credential_record = get_credential_record.sync(client=issuer_client, record_id=issuer_credential_record.record_id)

print(f"Issuer credential record: {issuer_credential_record.record_id}")
print(f"Holder credential record: {holder_credential_record.record_id}\n")

while(issuer_credential_record.protocol_state != IssueCredentialRecordAllOfProtocolState.REQUESTRECEIVED):
    issuer_credential_record = get_credential_record.sync(client=issuer_client, record_id=issuer_credential_record.record_id)
    holder_credential_record = get_credential_record.sync(client=holder_client, record_id=holder_credential_record.record_id)
    print(f"Issuer state: {issuer_credential_record.protocol_state} / Holder State: {holder_credential_record.protocol_state}")
    time.sleep(1)
    
print("\nIssuer credential record:\n")
print_credential_record(issuer_credential_record)

Please wait...
Issuer credential record: b02cafe0-a8e0-402b-9398-d0a6c3909b61
Holder credential record: 65601b27-13ad-4bec-8ba3-e5b51ee03f2e

Issuer state: OfferPending / Holder State: RequestPending
Issuer state: OfferSent / Holder State: RequestPending
Issuer state: RequestReceived / Holder State: RequestPending

Issuer credential record:

record_id:          b02cafe0-a8e0-402b-9398-d0a6c3909b61
subject_id:         did:prism:3b3cf4fc89942db5e84c4f4774c977368a8382f02d95b3f8ff6ff1bbdd44a4a6:Cs0CCsoCElkKBGtleTEQBEJPCglzZWNwMjU2azESILV-iwuC0ZbQCo4l0uJRGNhQ413e4YH2kkZYWaEseruLGiAooIpYsSFC3aTp7MwDdICS6JUw61vMzOZirbJOjUrbmBJZCgRrZXkyEAJCTwoJc2VjcDI1NmsxEiAcQuf8FYngKiJIvc1OxJ-BiuQ6h6n2OY2jRjx_YHWUoxogZB-tMdujutwiKKl5h_UpiuBuPu9xOxOhzh8PhXaVlx4SXAoHbWFzdGVyMBABQk8KCXNlY3AyNTZrMRIg3IHP3PMXes7WOiXbGt4i2xxmlt-YQxpSXLpxHvdwzmYaICjagobmCVU8J5YSYFjMNkDrUd3kaA2bTaZFHSU-wRCcGjQKD2RpZDpwcmlzbTp0ZXN0MRINTGlua2VkRG9tYWlucxoSaHR0cHM6Ly90ZXN0MS5jb20v
role:               Issuer
protocol_state:     RequestR

### Issuer - Accept credential request (issue credential to holder)

The Issuer uses the `issue_credential` endpoint to issue the credential. It must provide the `record_id`. When issued the record state changes to `CredentialSent` and the credential is send to the Holder.

In [12]:
issuer_credential_record: Response[IssueCredentialRecord] = issue_credential.sync(client=issuer_client, record_id=issuer_credential_record.record_id)
print("\nIssuer credential record:\n")
print_credential_record(issuer_credential_record)


Issuer credential record:

record_id:          b02cafe0-a8e0-402b-9398-d0a6c3909b61
subject_id:         did:prism:3b3cf4fc89942db5e84c4f4774c977368a8382f02d95b3f8ff6ff1bbdd44a4a6:Cs0CCsoCElkKBGtleTEQBEJPCglzZWNwMjU2azESILV-iwuC0ZbQCo4l0uJRGNhQ413e4YH2kkZYWaEseruLGiAooIpYsSFC3aTp7MwDdICS6JUw61vMzOZirbJOjUrbmBJZCgRrZXkyEAJCTwoJc2VjcDI1NmsxEiAcQuf8FYngKiJIvc1OxJ-BiuQ6h6n2OY2jRjx_YHWUoxogZB-tMdujutwiKKl5h_UpiuBuPu9xOxOhzh8PhXaVlx4SXAoHbWFzdGVyMBABQk8KCXNlY3AyNTZrMRIg3IHP3PMXes7WOiXbGt4i2xxmlt-YQxpSXLpxHvdwzmYaICjagobmCVU8J5YSYFjMNkDrUd3kaA2bTaZFHSU-wRCcGjQKD2RpZDpwcmlzbTp0ZXN0MRINTGlua2VkRG9tYWlucxoSaHR0cHM6Ly90ZXN0MS5jb20v
role:               Issuer
protocol_state:     CredentialPending
created_at:         2023-03-07 18:27:51.910417+00:00
updated_at:         2023-03-07 18:27:57.527857+00:00


### Holder - Wait for credential

The Holder waits to receive the credential. When received, it will be added to the Holder's credential record and the `protocol_state` will be updated to `CredentialReceived`. The code below waits until a credential is received.

In [14]:
print("Please wait...")

holder_credential_record = get_credential_record.sync(client=holder_client, record_id=holder_credential_record.record_id)

print(f"Issuer credential record: {issuer_credential_record.record_id}")
print(f"Holder credential record: {holder_credential_record.record_id}\n")

while(holder_credential_record.protocol_state != IssueCredentialRecordAllOfProtocolState.CREDENTIALRECEIVED):
    issuer_credential_record = get_credential_record.sync(client=issuer_client, record_id=issuer_credential_record.record_id)
    holder_credential_record = get_credential_record.sync(client=holder_client, record_id=holder_credential_record.record_id)
    print(f"Issuer state: {issuer_credential_record.protocol_state} / Holder State: {holder_credential_record.protocol_state}")
    time.sleep(1)

print("\nHolder credential record:\n")
print_credential_record(holder_credential_record)

Please wait...
Issuer credential record: b02cafe0-a8e0-402b-9398-d0a6c3909b61
Holder credential record: 65601b27-13ad-4bec-8ba3-e5b51ee03f2e


Holder credential record:

record_id:          65601b27-13ad-4bec-8ba3-e5b51ee03f2e
subject_id:         did:prism:3b3cf4fc89942db5e84c4f4774c977368a8382f02d95b3f8ff6ff1bbdd44a4a6:Cs0CCsoCElkKBGtleTEQBEJPCglzZWNwMjU2azESILV-iwuC0ZbQCo4l0uJRGNhQ413e4YH2kkZYWaEseruLGiAooIpYsSFC3aTp7MwDdICS6JUw61vMzOZirbJOjUrbmBJZCgRrZXkyEAJCTwoJc2VjcDI1NmsxEiAcQuf8FYngKiJIvc1OxJ-BiuQ6h6n2OY2jRjx_YHWUoxogZB-tMdujutwiKKl5h_UpiuBuPu9xOxOhzh8PhXaVlx4SXAoHbWFzdGVyMBABQk8KCXNlY3AyNTZrMRIg3IHP3PMXes7WOiXbGt4i2xxmlt-YQxpSXLpxHvdwzmYaICjagobmCVU8J5YSYFjMNkDrUd3kaA2bTaZFHSU-wRCcGjQKD2RpZDpwcmlzbTp0ZXN0MRINTGlua2VkRG9tYWlucxoSaHR0cHM6Ly90ZXN0MS5jb20v
role:               Holder
protocol_state:     CredentialReceived
created_at:         2023-03-07 18:27:52.752750+00:00
updated_at:         2023-03-07 18:28:00.999246+00:00


### JWT Credential

The JWT Credential is available in the holder credential record. The website https://jwt.io/ can be used to decode the credential.

In [15]:
print(base64.b64decode(holder_credential_record.jwt_credential).decode())

eyJhbGciOiJFUzI1NksifQ.eyJpc3MiOiJkaWQ6cHJpc206Y2U4N2ExNDBlMGI0M2JhMWZlYjliNTVlY2U2MjZiZDIyMjllMDU3OTA3YmQ0NmM1Zjg2NmI5NjczOGEzYjJmZTpDczBDQ3NvQ0Vsa0tCR3RsZVRFUUJFSlBDZ2x6WldOd01qVTJhekVTSVBGT2pGeGpWQWJFWEp5N1lLbS1iWlhzY1QwSkROdlF2dG9jOUdZSzJrTkFHaURfWlg2VE5GSllOcFBSRjRleWQtQkN5LWpnVk5wWVRWZzdMY0owVGxkQmNCSlpDZ1JyWlhreUVBSkNUd29KYzJWamNESTFObXN4RWlCd1JGaUZfMU01UGhGUjlrTDBMbHlkMjFhQkU1WVpDX0hJSkFRZjVoRERXQm9nZGQ3X2Y2S0J2Zno5cjhVbEstNlNzZWs5RFctUmZDWEkzaVR0OGE3dFZLNFNYQW9IYldGemRHVnlNQkFCUWs4S0NYTmxZM0F5TlRack1SSWdfeHFqb1paRVVQdFUtNllCdFo1NU9zaXRETXNQUFhEQ2puZ3h0MXY5R0hFYUlOZUNfekxQcTlscnhGV1pSSnhlQW1wNFY2T0hUQ1RxRHR3TTNhOG8xQlJpR2pRS0QyUnBaRHB3Y21semJUcDBaWE4wTVJJTlRHbHVhMlZrUkc5dFlXbHVjeG9TYUhSMGNITTZMeTkwWlhOME1TNWpiMjB2Iiwic3ViIjoiZGlkOnByaXNtOjNiM2NmNGZjODk5NDJkYjVlODRjNGY0Nzc0Yzk3NzM2OGE4MzgyZjAyZDk1YjNmOGZmNmZmMWJiZGQ0NGE0YTY6Q3MwQ0Nzb0NFbGtLQkd0bGVURVFCRUpQQ2dselpXTndNalUyYXpFU0lMVi1pd3VDMFpiUUNvNGwwdUpSR05oUTQxM2U0WUgya2taWVdhRXNlcnVMR2lBb29JcFlzU0ZDM2FUcDdNd0RkSUNTNkpVdzYxdk16T

### ⚠️ Important Note
Keep the Holder credential record identifier at hand, it will be needed to run the next example:

In [16]:
print(f"\nHolder credential record_id: {holder_credential_record.record_id}\n")


Holder credential record_id: 65601b27-13ad-4bec-8ba3-e5b51ee03f2e

